## Build a Wflow model from scratch

This notebook demonstrates how to prepare **Wflow** model from scratch using the command line interace (CLI).

All lines in this notebook which start with `!` are executed from the command line. Within the notebook environment the logging messages are shown after completion. You can also copy these lines and paste these in your shell to get more direct feedback.

### HydroMT CLI build interface

Lets first check if the Wflow model is recognized by HydroMT

In [ ]:
# this should return "wflow, wflow_sediment"
# as well as the generic HydroMT models "grid_model, lumped_model, network_model, (mesh_model)"
!hydromt --models

Using the **HydroMT build** API we can setup a complete model from scratch. Let's get an overview of all the available options:

In [ ]:
!hydromt build --help

### Model setup configuration

The HydroMT configuration file (YAML) contains the model setup configuration and determines which methods are used to prepare the different components of a Wflow model and in which order and optionally sets non-default arguments for each method. This configuration is passed to HydroMT using `-i <path_to_config_file>`. We have prepared several example files which are available in the model repository [examples folder](https://github.com/Deltares/hydromt_wflow/tree/main/examples) and from the [docs (building a model)](https://deltares.github.io/hydromt_wflow/latest/user_guide/wflow_build). 

Each header (without indent) (e.g. `setup_basemaps:`) corresponds to a model method. All model methods are explained in the [docs (model components)](https://deltares.github.io/hydromt_wflow/latest/user_guide/wflow_model_setup.html). 

We will load the default wflow build configuration yaml file for inspection:

In [ ]:
fn_config = "wflow_build.yml"
with open(fn_config, "r") as f:
    txt = f.read()
print(txt)

### Data for model setup

The `<...>_fn` arguments correspond to a data source from the `DataCatalog` which is based on a yml-file with references to the data paths/urls and how the data should be read. This file can be provided to hydromt using `-d <path_to_yml_file>`. 

By default, the pre-defined `artifact_data` catalog containing some example data for the Piave basin will be downloaded to `~/.hydromt_data/` which is also used for this example. An overview of the available example data is provided [here](https://deltares.github.io/hydromt/latest/user_guide/data_existing_cat.html)
This example data is a based on the data which is available from the Deltares p-drive. 
If you have acces to this drive, a pre-configured catalog file can be loaded using the `-d deltares_data` pre-defined catalog. 

More background how to write a data catalog yml file can be found in the [HydroMT core docs](https://deltares.github.io/hydromt/latest/user_guide/data_prepare_cat.html) 

### Setup complete Wflow model 

Once we have prepared our data catalog and model build settings file, we are ready to ask HydroMT to build our Wflow model. Building the model can be done from the command line. Down below is an example for the Northern part of the Piave basin in Italy:

In [ ]:
# NOTE: copy this line (without !) to your shell for more direct feedback
!hydromt build wflow "./wflow_test_full" -r "{'subbasin': [12.2051, 45.8331], 'strord': 4, 'bounds': [11.70, 45.35, 12.95, 46.70]}" -i wflow_build.yml -d artifact_data -vv

With this example we build a complete Wflow model including forcing. The example means the following: run **hydromt build** with:

* `wflow` : i.e. build a Wflow model
* `./wflow_test_full` : output model folder
* `-r "{'subbasin': [12.2051, 45.8331], 'strord': 4, 'bounds': [11.70, 45.35, 12.95, 46.70]}"` : derive a subbasin with its outlet at the given x,y coordinates and within the given bounds [xmin, ymin, xmax, ymax] (WGS84) snapped to a river with minimum stream order (strord) of 4. All *REGION* options are described in the [docs](https://deltares.github.io/hydromt/latest/user_guide/model_region)
* `-i wflow_build.yml` : setup configuration file including all components to build and their arguments
* `-d artifact_data`: data catalog to use. Here `artifact_data` which contains a global data extract for Northern Italy for demo purposes.
* `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.


Next we check which files have been created. The model root should contain two netcdf (.nc) files, one for the staticmaps and one for the forcing data, a wflow run configuration (.toml) file. These files are sufficient to run the wflow model. In addition several geometry (.geojson) files are saved in the *staticgeoms* folder.  Finally, the setup process is logged in the hydromt.log file. 

In [ ]:
import os

root = "wflow_test_full"
for path, _, files in os.walk(root):
    print(path)
    for name in files:
        if name.endswith(".xml"):
            continue
        print(f" - {name}")

### Visualize and/or inspect model schematization

* The **wflow plot** example notebook contains scripts to visualize your model
* The **wflow nc to raster files** example notebook contains scripts to write the nc model files to a netcdf which can be used to inspect (and modify) the model layers in e.g. QGIS.